Basic test for Nvidia DALI

# Import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torch.optim as optim
from IPython.core.debugger import set_trace
import pandas as pd
import lmdb
import pickle
import h5py
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIGenericIterator

RuntimeError: CUDA runtime API error cudaErrorUnknown (30):
unknown error

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Config

In [ ]:
path_data = Path('data')

In [ ]:
!ls {path_data}

# Networks

In [ ]:
class FastNet(nn.Module):
    # Just do a single convolution followed by a linear layer
    # Made to be simple to emphasize affect of image loading
    # and augmentation
    def __init__(self, num_cl):
        super(FastNet, self).__init__()
        self.conv = nn.Conv2d(3, 64, 3, stride=2)
        self.fc = nn.Linear(64, num_cl)
        
    def forward(self, x):
        x = self.conv(x)
        x = F.adaptive_avg_pool2d(x, 1)
        x = torch.flatten(x, 1)
        x = self.fc(x)        
        return x

# Pipelines

In [ ]:
# jpg no augmentation pipeline
class ImgNaPipeline(Pipeline):
    def __init__(self, path_imgs, num_batch, num_threads, device_id):
        super(ImgNaPipeline, self).__init__(num_batch, num_threads, device_id)
        self.input = ops.FileReader(file_root=path_imgs, 
                                    random_shuffle=True)
        self.decode = ops.ImageDecoder(device='mixed', 
                                       output_type=types.RGB)
        self.cmnp = ops.CropMirrorNormalize(device="gpu",
                                            output_dtype=types.FLOAT,
                                            output_layout=types.NCHW,
                                            image_type=types.RGB,
                                            mean=[0.5 * 255,0.5 * 255,0.5 * 255],
                                            std= [0.5 * 255,0.5 * 255,0.5 * 255])

    def define_graph(self):
        jpgs, labels = self.input()
        imgs = self.decode(jpgs)    # Images are decoded on gpu
        imgs = self.cmnp(imgs)      # This also converts from NHWC -> NCHW
        return (imgs, labels)

# Losses

In [ ]:
loss = nn.CrossEntropyLoss()

# Train

In [ ]:
def train(di, model, loss, opt, num_epochs):
    for epoch in range(num_epochs):
        for data in di:
            X = data[0]["data"]
            y = data[0]["label"].squeeze().cuda().long()            
            
            opt.zero_grad()    # Zero gradients
            y_hat = model(X)   # Forward pass
            l = loss(y_hat, y) # Loss
            l.backward()       # Compute gradients
            opt.step()         # Step

        # print statistics
        print(f'Epoch: {epoch}; Loss: {l.item()}')

# Test Images

### Small size

In [ ]:
pipe = ImgNaPipeline(path_imgs=path_data/'small_imgs'/'png', 
                     num_batch=256, 
                     num_threads=12, 
                     device_id=0)

pipe.build()

di = DALIGenericIterator(pipe, 
                         ['data', 'label'], 
                         40000)

In [ ]:
model = FastNet(10).cuda()
opt = optim.SGD(model.parameters(), lr=0.001)
%time train(di, model, loss, opt, 1)

Super fast...

### Medium size

In [ ]:
pipe = ImgNaPipeline(path_imgs=path_data/'medium_imgs'/'png', 
                     num_batch=64, 
                     num_threads=12, 
                     device_id=0)

pipe.build()

di = DALIGenericIterator(pipe, 
                         ['data', 'label'], 
                         40000)

In [ ]:
model = FastNet(10).cuda()
opt = optim.SGD(model.parameters(), lr=0.001)
%time train(di, model, loss, opt, 1)

So fast that I'm convinced I have a bug somewhere...